# Site Monitoring and Contour Generation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import json
import os

import cv2
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rs
import shapely
from tensorflow import keras
from tqdm import tqdm

from scripts import dl_utils
from scripts.dl_utils import predict_spectrogram, rect_from_point
from scripts.nn_predict import make_predictions, visualize_predictions

## Download Data

### Download Single Site

In [ ]:
def generate_contours(preds, dates, threshold=0.3, plot=False):
    # Set a prediction threshold. Given that the heatmaps are blurred, it is recommended
    # to set this value lower than you would in blob detection
    contour_list = []
    date_list = []
    for pred, date in zip(preds, dates):
        masked_percentage = np.sum(pred.mask / np.size(pred.mask))
        if masked_percentage < 0.1:
            input_img = (pred* 255).astype(np.uint8)
            blurred = cv2.GaussianBlur(input_img, (3,3), cv2.BORDER_DEFAULT)
            ret, thresh = cv2.threshold(blurred, int(threshold * 255), 255, cv2.THRESH_TOZERO)
            # Note that cv2.RETR_CCOMP returns a hierarchy of parent and child contours
            # Needed for fixing the case with polygon holes 
            # https://docs.opencv.org/master/d9/d8b/tutorial_py_contours_hierarchy.html
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
            contour_list.append(contours)
            date_list.append(date)
            if plot:
                plt.figure(figsize=(16,4), dpi=150)
                plt.subplot(1,4,1)
                plt.imshow(input_img, vmin=0, vmax=255)
                plt.title('Pred')
                plt.axis('off')
                plt.subplot(1,4,3)
                plt.imshow(thresh, vmin=0, vmax=255)
                plt.title('Thresholded Blur')
                plt.axis('off')
                plt.subplot(1,4,2)
                plt.imshow(blurred, vmin=0, vmax=255)
                plt.title('Blurred')
                plt.axis('off')
                plt.subplot(1,4,4)
                three_channel_preds = np.stack((pred * 255, pred * 255, pred * 255), axis=-1)
                preds_contour_img = cv2.drawContours(three_channel_preds, contours, -1, (255, 0, 0), 1)
                plt.imshow(preds_contour_img /255, vmin=0, vmax=255)
                plt.title(f"{len(contours)} separate contours")
                plt.axis('off')
                plt.suptitle(date)
                plt.show()
    return contour_list, date_list

def generate_polygons(contour_list, bounds_list, plot=False):
    contour_multipolygons = []
    for contours, bounds in zip(contour_list, bounds_list):
        # Define patch coordinate bounds to set pixel scale
        bounds = shapely.geometry.Polygon(bounds).bounds
        transform = rs.transform.from_bounds(*bounds, mosaics[0].shape[0], mosaics[0].shape[1])
        polygon_coords = []
        for contour in contours:
            # Convert from pixels to coords
            contour_coords = []
            for point in contour[:,0]:
                lon, lat = rs.transform.xy(transform, point[1], point[0])
                contour_coords.append([lon, lat])
            if len(contour_coords) > 1:
                # Close the loop
                contour_coords.append(contour_coords[0])
                # Add individual contour to list of contours for the month
                polygon_coords.append(contour_coords)

        contour_polygons = []
        for coord in polygon_coords:
            poly = shapely.geometry.Polygon(coord)
            # A single line of pixels will be recognized as a line rather than a polygon
            # Inflate the area by a small amount to create a polygon
            if poly.area == 0:
                poly = poly.buffer(0.00002)
            contour_polygons.append(poly)
        multipolygon = shapely.geometry.MultiPolygon(contour_polygons)
        # Currently, "holes" in a polygon are seen as separate contours.
        # This means that there will be overlapping polygons. Shapely can 
        # detect this case, but can't fix it automatically. To rectify, the
        # unary_union operator and .buffer(0) hack removes interior polygons.
        if not multipolygon.is_valid:
            multipolygon = shapely.ops.unary_union(multipolygon).buffer(0)
        if plot:
            display(multipolygon)
        contour_multipolygons.append(multipolygon)
    return contour_multipolygons

In [ ]:
RECT_WIDTH = 0.008
START_DATE = '2019-06-01'
END_DATE = '2021-09-01'
MOSAIC_PERIOD = 3
SPECTROGRAM_INTERVAL = 2

In [ ]:
model_name = 'spectrogram_v0.0.8_2021-06-03'
model = keras.models.load_model(f'../models/{model_name}.h5')

In [ ]:
confirmed_sites_file = 'v12_java_validated_positives'
confirmed_sites = gpd.read_file(f"../data/sampling_locations/{confirmed_sites_file}.geojson")
display(confirmed_sites.head())
coords = [[site.x, site.y] for site in confirmed_sites['geometry']]
names = [site for site in confirmed_sites['name']]

In [ ]:
contour_gdf = gpd.GeoDataFrame(columns=['geometry', 'area (km^2)', 'date', 'name']).set_crs('EPSG:4326')
for coord, name in zip(coords, names):
    print(name)
    mosaics, metadata = dl_utils.download_mosaics(
        rect_from_point(coord, RECT_WIDTH), START_DATE, END_DATE, MOSAIC_PERIOD, method='min')
    pairs = dl_utils.pair(mosaics, SPECTROGRAM_INTERVAL)
    preds = [predict_spectrogram(pair, model) for pair in pairs]
    dates = dl_utils.get_starts(START_DATE, END_DATE, 3, 2)[SPECTROGRAM_INTERVAL:]
    bounds = [sample['wgs84Extent']['coordinates'][0][:-1] for sample in metadata[SPECTROGRAM_INTERVAL:]]
    contours, contour_dates = generate_contours(preds, dates, threshold=0.5, plot=False)
    polygons = generate_polygons(contours, bounds, plot=False)
    gdf = gpd.GeoDataFrame(geometry=polygons).set_crs('EPSG:4326')
    gdf['date'] = [datetime.datetime.fromisoformat(date) for date in contour_dates]
    gdf['area (km^2)'] = gdf['geometry'].to_crs('epsg:3395').map(lambda p: p.area / 10**6)
    gdf['name'] = [name for _ in range(len(contour_dates))]
    contour_gdf = contour_gdf.append(gdf)
contour_gdf.to_file(f'../data/model_outputs/site_contours/{confirmed_sites_file}_contours_model_{model_name}.geojson', driver='GeoJSON')